In [1]:
import tensorflow as tf
import numpy as np

In [21]:
a=tf.constant(10)
b=tf.constant(2)
c=tf.div(a, b)
#init_op=tf.global_variables_initializer()

In [22]:
with tf.Session() as sess:
    print a.eval()
    sess.run(init_op)
    print(sess.run(c))

10
5


## network variable_name test

In [12]:
x=tf.constant(1)
w=tf.constant(2)
o=tf.multiply(x,w)
o=tf.multiply(o,w)
o=tf.multiply(o,w)
o=tf.multiply(o,w)
print o,w
with tf.Session() as sess:
    tf.global_variables_initializer()
    r=sess.run(o)
    print r

Tensor("Mul_15:0", shape=(), dtype=int32) Tensor("Const_9:0", shape=(), dtype=int32)
16


## multi returns

In [5]:
a=tf.constant(1)
b=tf.constant(2)
o1=tf.multiply(a,b)
o2=tf.multiply(o1,b)
o3=tf.multiply(o2,b)
with tf.Session() as sess:
    # how many times does o1=tf.muliply run?
    print(sess.run([o1,o2,o3]))

[2, 4, 8]


In [10]:
hello=tf.constant('hello')
with tf.Graph().as_default():
    hi=tf.constant("hi")
    print(hi.graph)
    print(hello.graph)